In [1]:
import pandas as pd # data processing
import numpy as np # working with arrays
import matplotlib.pyplot as plt # visualization
import seaborn as sns
import random
from termcolor import colored as cl # text customization

In [3]:
df_full = pd.read_excel('C:/Users/KABYADE/Desktop/Fraud_ML/fraud-detection/data_preparation/Authorization_data_v2.xlsx')

In [5]:
columns_num = df_full.select_dtypes(include=np.number).columns.tolist()
columns_num.remove('CLASS')
max_values = {}

for v in columns_num:
    max_values[v] = df_full[v].max()


In [8]:
# Load the credit card transaction data   
df = pd.read_excel('C:/Users/KABYADE/Desktop/Fraud_ML/fraud-detection/data_preparation/data/Khadija_2.xlsx')
df.head(1)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,CLASS
0,1,200,22.0,10112.0,500,01/06/2009,01/06/2011,11/07/2009 09:06:38,2000.0,2000.0,...,NaN,0.0,1000001,1000001,26/12/2009,2000.0,952,384,VERSUS BANK PLATEAU REGION LAGUNECI,1


In [12]:
df.loc[df['CLASS']==0 , 'CLASS'] = 1
#len(df[df.CLASS == 1])
len(df)

9891

In [13]:
#Split data_to upate into 3 parts
dfs = np.array_split(df, 7)
R_1,R_2,R_3,R_4,R_5,R_6,R_7,=dfs

print(len(R_1))
print(len(R_2))

1413
1413


In [14]:
def random_not_in(l, max):
    res = l[0]
    while(res in l):
        res = random.randint(0, max)
    return res

In [16]:
""" R1
V4 = ''000010''
V1 IN (''00'')
V10 BETWEEN 999.99  AND 7999.99 )
V16 LIKE ''5___________''
(V16,7,1)   IN  (''2'',''8'')
""" 
Count_R_1=len(R_1)
V10_values = [random.uniform(999.99,7999.99) for i in range(Count_R_1)]
#print(len(V10_values))

R_1['V4'] = '000010'
R_1['V1'] = '00'
R_1['V10'] = V10_values
R_1['V16']='5'+R_1['V16'].str[1:] 

for i in range(len(R_1)):
    R_1.at[i, 'V16'] = R_1.at[i, 'V16'][:7] + random.choice(['2', '8']) + R_1.at[i, 'V16'][8:]


In [17]:
""" R2
V4 = ''000010''
V1 IN (''00'')
V13  NOT IN (''7995'')
V10 >= 60000
(V16,7,1)   IN  (''5'')
""" 

R_2['V4'] = '000010'
R_2['V1'] = '00'
R_2['V13'] = random_not_in([7995], max_values['V13']) 
R_2['V10'] = random.randint(60000, max_values['V10'])
R_2['V16']=R_2['V16'].str.slice_replace(7, 8, '5')


In [18]:
""" R3
V4 = ''000010''
(V16,7,1)   NOT IN  (''5'')
""" 

R_3['V4'] = '000010'
for index, row in R_3.iterrows():
    R_3.at[index, 'V16'] = row['V16'][:7] +  str(random_not_in([5], 9))  + row['V16'][8:]


In [20]:
""" R4
V4 = ''000010''
V1 IN (''01'')
V10 < 100
""" 

R_4['V4'] = '000010'
R_4['V1'] = '01'
R_4['V10'] = random.randint(0, 99)


In [21]:
""" R5
V4 = ''000010''
V1 IN (''01'')
V26 IN (''192'')
V21  IN (''00000008'')
""" 

R_5['V4'] = '000010'
R_5['V1'] = '01'
R_5['V26'] = 192
R_5['V21'] = '00000008'


In [22]:
""" R6
V4 = ''000010''
V1 IN (''01'')
V26 IN (''360'',''508'')
V15 IN (''90'')
""" 

R_6['V4'] = '000010'
R_6['V1'] = '01'
R_6['V26'] =  np.random.choice([360, 508],size=len(R_6['V26']))
R_6['V15'] = 90


In [23]:
""" R7
V4 = ''000010'' 
V26 NOT IN (''072'',''426'',''508'',''516'') 
V12  NOT IN (''001264'') 
V15 IN (''90'')
""" 

R_7['V4'] = '000010'
R_7['V26'] = np.random.choice(random_not_in([72,426,508,516], max_values['V26']) ,size=len(R_7['V26']))
R_7['V12'] = random_not_in([1264], max_values['V12']) 
R_7['V15'] = 90


In [24]:
df_final_Part1 = pd.concat([R_1,R_2,R_3,R_4,R_5,R_6,R_7])

In [26]:
len(df_final_Part1)
df_final_Part1.to_excel('output/khadija_fraud_2.xlsx', index=False)